Use what you have learned in your previous solutions to read in the new york rodent inspection data as a pandas dataframe, and then augment that dataframe with wait time (interval between inspection and approval), weekday, and iso-week of inspection. also add a new column, season of the inspection date (defined as: dec-feb, inclusive, is winter, mar-may is spring, jun-aug is summer, sep-nov is fall).

then answer the following questions:

between the dates of 2010-01-01 and 2010-03-31, what was the daily average number of inspections?
between the dates of 2010-01-01 and 2010-03-31, which weekday had the greatest number of inspections?
between the dates of 2010-01-01 and 2010-03-31, which weekday had the least number of inspections?
if you are confident that you can already solve this, the rest of the session is not much use to you.

as a bonus we should be able to answer these further questions

which weekday has the longest average wait time for approval in the winter? (let's define the seasons as: (dec-feb is winter, mar-may is spring, jun-aug is summer, sep-nov is fall).
which weekday has the longest average wait time for approval in the summer?
which season has the greatest number of inspections?
which season has the greatest number of distinct dates ...
a) in the data set
b) in the calendar?
which borough has the greatest difference in the number of inspections in the spring vs in the fall?
count the number of inspections per iso-week. find the week with the greatest number of inspections. for that week, and that week only, count the inspections by day-of-week.

In [1]:
import datetime
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir('../../')
print(os.listdir('.'))

['.DS_Store', 'old_bk_gitconfig', 'solutions', 'code snippets', 'README.md', 'Pipfile', 'notes', '.gitignore', 'exercises', '.git', 'Pipfile.lock', 'using_apis']


In [3]:
os.chdir('exercises/04_dataframes') 
print(os.listdir('.'))

['NY_rodent_inspections_sample.csv', 'NY_rodent_inspections_sample_medium.csv', 'pandas_python_cheat_sheet.pdf', 'Pandas_Cheat_Sheet.pdf', '.ipynb_checkpoints', 'dataframes.ipynb', 'NY_rodent_inspections_sample_small.csv']


In [4]:
csv_name = 'NY_rodent_inspections_sample.csv'
rodent_dataframe = pd.read_csv(csv_name)
rodent_dataframe.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,10023,990505,223527,40.780204,-73.977414,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,10023,990516,223521,40.780188,-73.977375,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,10467,1020110,252216,40.858877,-73.870364,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.831321,-73.861994,Bronx,11/09/2009 11:10:16 AM,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)"
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,10024,989641,224567,40.783059,-73.980533,Manhattan,11/10/2009 08:40:42 AM,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)"


In [5]:
def format_iso_week(datetime_object):
    the_date = datetime_object.date()
    iso_year, iso_week, iso_weekday = the_date.isocalendar()
    iso_week_string = str(iso_year) + '-W' + str(iso_week)
    return iso_week_string

In [6]:
original_format = '%m/%d/%Y %I:%M:%S %p'

In [7]:
rodent_dataframe['inspection_datetime']   = rodent_dataframe.apply(lambda row: datetime.datetime.strptime(row['INSPECTION_DATE'], original_format), axis=1)

rodent_dataframe['approval_datetime']     = rodent_dataframe.apply(lambda row: datetime.datetime.strptime(row['APPROVED_DATE'],   original_format), axis=1)



rodent_dataframe['inspection_weekday']    = rodent_dataframe.apply(lambda row: row['inspection_datetime'].strftime("%A"), axis=1)

rodent_dataframe['inspection_month']      = rodent_dataframe.apply(lambda row: row['inspection_datetime'].strftime("%B"), axis=1)

rodent_dataframe['isoweek']               = rodent_dataframe.apply(lambda row: format_iso_week(row['inspection_datetime']), axis=1)

rodent_dataframe['days_taken_for_approval'] = rodent_dataframe.apply(lambda row: (row['approval_datetime'].date() - row['inspection_datetime'].date()), axis=1)

rodent_dataframe['just_inspection_date'] = rodent_dataframe.apply(lambda row: (row['approval_datetime'].date()), axis=1)

In [8]:
rodent_dataframe.loc[(rodent_dataframe.inspection_month == 'December') | (rodent_dataframe.inspection_month == 'January') | (rodent_dataframe.inspection_month == 'Febuary'), 'season'] = 'Winter'   
rodent_dataframe.loc[(rodent_dataframe.inspection_month == 'March') | (rodent_dataframe.inspection_month == 'April') | (rodent_dataframe.inspection_month == 'May'), 'season'] = 'Spring'
rodent_dataframe.loc[(rodent_dataframe.inspection_month == 'June') | (rodent_dataframe.inspection_month == 'July') | (rodent_dataframe.inspection_month == 'August'), 'season'] = 'Summer'
rodent_dataframe.loc[(rodent_dataframe.inspection_month == 'September') | (rodent_dataframe.inspection_month == 'October') | (rodent_dataframe.inspection_month == 'November'), 'season'] = 'Autumn'

In [9]:
rodent_dataframe.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,APPROVED_DATE,LOCATION,inspection_datetime,approval_datetime,inspection_weekday,inspection_month,isoweek,days_taken_for_approval,just_inspection_date,season
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,...,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)",2009-10-14 12:00:27,2009-10-14 15:01:46,Wednesday,October,2009-W42,0 days,2009-10-14,Autumn
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,...,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)",2009-10-14 12:51:21,2009-10-14 15:02:30,Wednesday,October,2009-W42,0 days,2009-10-14,Autumn
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,...,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)",2009-11-09 12:59:55,2009-11-10 14:54:52,Monday,November,2009-W46,1 days,2009-11-10,Autumn
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,...,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)",2009-11-09 11:10:16,2009-11-10 14:56:42,Monday,November,2009-W46,1 days,2009-11-10,Autumn
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,...,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)",2009-11-10 08:40:42,2009-11-17 11:39:11,Tuesday,November,2009-W46,7 days,2009-11-17,Autumn


In [10]:
earliest_date = rodent_dataframe['inspection_datetime'].min()
latest_date = rodent_dataframe['inspection_datetime'].max()
print('the inspection dates range from', earliest_date, 'to', latest_date)

the inspection dates range from 2009-01-16 10:04:24 to 2011-10-27 18:28:10


In [11]:
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2010-03-31')
criteria = (rodent_dataframe.inspection_datetime >= start_date) & (rodent_dataframe.inspection_datetime <= end_date)
print('there are', sum(criteria), 'rows selected')

there are 3406 rows selected


In [12]:
filtered_rodent_df = rodent_dataframe[criteria]

In [13]:
# 1) what was the daily average number of inspections?

filtered_rodent_df\
    [['just_inspection_date']]\
    .groupby('just_inspection_date')\
    .agg({'just_inspection_date':'count'})['just_inspection_date']\
    .describe()

count     68.000000
mean      50.088235
std       42.624264
min        1.000000
25%       19.000000
50%       38.000000
75%       71.250000
max      167.000000
Name: just_inspection_date, dtype: float64

In [14]:
# 2) which weekday had the greatest number of inspections? 3) which weekday had the least number of inspections?

# we often want to calculate summary statistics of similar sections dataframe, we do this by first grouping by a column value, and then aggregating the values in each group. e.g. here we might want to know how many inspections there are in each borough. to do that we need to group by the borough column values and aggregating all the rows for each borough with the count() function.

# access a list of columns

filtered_rodent_df[['inspection_weekday']].groupby('inspection_weekday').agg({'inspection_weekday':'count'})


,inspection_weekday
inspection_weekday,
Friday,671
Monday,658
Sunday,1
Thursday,739
Tuesday,795
Wednesday,542
